In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from tqdm import tqdm
from scipy import stats
pd.set_option('display.max_columns', 100)

In [2]:
df = dd.read_csv('data/*matches*.csv', dtype={'home_team_goal_timings': 'object', 
                                             'away_team_goal_timings': 'object', 'Game Week': 'float64'})

In [3]:
df_players = dd.read_csv('data/*players*.csv', dtype={'season': 'object'})

In [4]:
df = df.compute()

In [5]:
df_players = df_players.compute()
df_players.dropna(subset=['full_name'], inplace=True)

In [6]:
df_matches_euro = pd.read_csv('euro-data/fixtures-1.csv')
df_teams_euro = pd.read_csv('euro-data/teams-1.csv')
df_players_euro = pd.read_csv('euro-data/players-active-1.csv')

In [7]:
df_matches_euro.head()

,mId,gmIsCurrent,gmIsLocked,gdId,isFeedLive,isLive,dateTime,dateTimeLock,htId,htName,htShortName,htCCode,atId,atName,atShortName,atCCode,gameNo,mdName,htScore,atScore,htAggScore,atAggScore,aggDescription,aggFlag,matchStatus,teamSc,teamScStartDate,teamScEndDate,lineupAnnounced,isMatchPostponed,groupId,groupName,stadiumId,stadiumName,stadiumThumb,venueId,venueName,venueCountryCode,mdId
0,2024447,2,1,1,2,2,06/11/2021 21:00:00,06/11/2021 21:00:00,135,Turkey,Turkey,TUR,66,Italy,Italy,ITA,33673,Match 1,0.0,3.0,0,0,NaN,0,2,NaN,NaN,NaN,1,0,2006438,Group A,57775,Olimpico in Rome,https://img.uefa.com/imgml/stadium/matchinfo/w...,2637,Rome,ITA,1
1,2024448,2,1,2,2,2,06/12/2021 15:00:00,06/12/2021 15:00:00,144,Wales,Wales,WAL,128,Switzerland,Switzerland,SUI,33673,Match 2,1.0,1.0,0,0,NaN,0,2,NaN,NaN,NaN,1,0,2006438,Group A,250002745,Baku Olympic Stadium,https://img.uefa.com/imgml/stadium/matchinfo/w...,1162,Baku,AZE,1
2,2024449,2,1,2,2,2,06/12/2021 18:00:00,06/12/2021 18:00:00,35,Denmark,Denmark,DEN,42,Finland,Finland,FIN,33673,Match 3,0.0,1.0,0,0,NaN,0,2,NaN,NaN,NaN,1,0,2006439,Group B,63462,Parken Stadium,https://img.uefa.com/imgml/stadium/matchinfo/w...,1449,Copenhagen,DEN,1
3,2024450,2,1,2,2,2,06/12/2021 21:00:00,06/12/2021 21:00:00,13,Belgium,Belgium,BEL,57451,Russia,Russia,RUS,33673,Match 4,3.0,0.0,0,0,NaN,0,2,NaN,NaN,NaN,1,0,2006439,Group B,250003363,Saint Petersburg Stadium,https://img.uefa.com/imgml/stadium/matchinfo/w...,2850,St Petersburg,RUS,1
4,2024451,2,1,3,2,2,06/13/2021 15:00:00,06/13/2021 15:00:00,39,England,England,ENG,56370,Croatia,Croatia,CRO,33673,Match 6,1.0,0.0,0,0,NaN,0,2,NaN,NaN,NaN,1,0,2006441,Group D,1100043,Wembley Stadium,https://img.uefa.com/imgml/stadium/matchinfo/w...,2116,London,ENG,1


In [8]:
df_matches_euro.tail(20)

,mId,gmIsCurrent,gmIsLocked,gdId,isFeedLive,isLive,dateTime,dateTimeLock,htId,htName,htShortName,htCCode,atId,atName,atShortName,atCCode,gameNo,mdName,htScore,atScore,htAggScore,atAggScore,aggDescription,aggFlag,matchStatus,teamSc,teamScStartDate,teamScEndDate,lineupAnnounced,isMatchPostponed,groupId,groupName,stadiumId,stadiumName,stadiumThumb,venueId,venueName,venueCountryCode,mdId
31,2024472,2,1,12,2,2,06/22/2021 21:00:00,06/22/2021 21:00:00,56370,Croatia,Croatia,CRO,117,Scotland,Scotland,SCO,33675,Match 32,3.0,1.0,0,0,NaN,0,2,NaN,NaN,NaN,1,0,2006441,Group D,62427,Hampden Park,https://img.uefa.com/imgml/stadium/matchinfo/w...,1701,Glasgow,SCO,3
32,2024473,2,1,13,2,2,06/23/2021 18:00:00,06/23/2021 18:00:00,58836,Slovakia,Slovakia,SVK,122,Spain,Spain,ESP,33675,Match 34,0.0,5.0,0,0,NaN,0,2,NaN,NaN,NaN,1,0,2006442,Group E,74459,La Cartuja Stadium,https://img.uefa.com/imgml/stadium/matchinfo/w...,2769,Seville,ESP,3
33,2024474,2,1,13,2,2,06/23/2021 18:00:00,06/23/2021 18:00:00,127,Sweden,Sweden,SWE,109,Poland,Poland,POL,33675,Match 35,3.0,2.0,0,0,NaN,0,2,NaN,NaN,NaN,1,0,2006442,Group E,250003363,Saint Petersburg Stadium,https://img.uefa.com/imgml/stadium/matchinfo/w...,2850,St Petersburg,RUS,3
34,2024476,2,1,13,2,2,06/23/2021 21:00:00,06/23/2021 21:00:00,110,Portugal,Portugal,POR,43,France,France,FRA,33675,Match 33,2.0,2.0,0,0,NaN,0,2,NaN,NaN,NaN,1,0,2006443,Group F,250004078,Puskás Aréna,https://img.uefa.com/imgml/stadium/matchinfo/w...,1327,Budapest,HUN,3
35,2024475,2,1,13,2,2,06/23/2021 21:00:00,06/23/2021 21:00:00,47,Germany,Germany,GER,57,Hungary,Hungary,HUN,33675,Match 36,2.0,2.0,0,0,NaN,0,2,NaN,NaN,NaN,1,0,2006443,Group F,85441,Football Arena Munich,https://img.uefa.com/imgml/stadium/matchinfo/w...,2298,Munich,GER,3
36,2024478,1,0,14,0,0,06/26/2021 18:00:00,06/26/2021 18:00:00,144,Wales,Wales,WAL,35,Denmark,Denmark,DEN,33676,Match 37,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,0,NaN,62417,Johan Cruijff ArenA,https://img.uefa.com/imgml/stadium/matchinfo/w...,1074,Amsterdam,NED,4
37,2024477,1,0,14,0,0,06/26/2021 21:00:00,06/26/2021 21:00:00,66,Italy,Italy,ITA,8,Austria,Austria,AUT,33676,Match 38,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,0,NaN,1100043,Wembley Stadium,https://img.uefa.com/imgml/stadium/matchinfo/w...,2116,London,ENG,4
38,2024480,0,0,15,0,0,06/27/2021 18:00:00,06/27/2021 18:00:00,95,Netherlands,Netherlands,NED,58837,Czech Republic,Czech Republic,CZE,33676,Match 39,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,0,NaN,250004078,Puskás Aréna,https://img.uefa.com/imgml/stadium/matchinfo/w...,1327,Budapest,HUN,4
39,2024479,0,0,15,0,0,06/27/2021 21:00:00,06/27/2021 21:00:00,13,Belgium,Belgium,BEL,110,Portugal,Portugal,POR,33676,Match 40,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,0,NaN,74459,La Cartuja Stadium,https://img.uefa.com/imgml/stadium/matchinfo/w...,2769,Seville,ESP,4
40,2024482,0,0,16,0,0,06/28/2021 18:00:00,06/28/2021 18:00:00,56370,Croatia,Croatia,CRO,122,Spain,Spain,ESP,33676,Match 41,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,0,NaN,63462,Parken Stadium,https://img.uefa.com/imgml/stadium/matchinfo/w...,1449,Copenhagen,DEN,4


In [9]:
df_teams_euro.head()

,id,webName,offName,shortName,typeIsNational,countryCode,isEliminated
0,8,Austria,Austria,AUT,1,AUT,NaN
1,13,Belgium,Belgium,BEL,1,BEL,NaN
2,35,Denmark,Denmark,DEN,1,DEN,NaN
3,39,England,England,ENG,1,ENG,NaN
4,43,France,France,FRA,1,FRA,NaN


In [10]:
df_players_euro.head()

,id,pDName,pFName,latinName,tName,tId,teamPlayed,cCode,skill,value,isActive,selPer,mdId,totPts,gS,assist,cS,gC,yC,rC,oG,pS,pC,pE,saves,pM,bR,gOB,mOM,mOMPts,pStatus,matchAtd,trained,isPlayed,selInPer,selOutPer,upcomingMatchesList,currentMatchesList,avgPlayerPts,avgPlayerValue,lastGdPoints,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10,category11,category12,category13,category14,category15
0,63706,C. Ronaldo,Cristiano Ronaldo,Cristiano Ronaldo,Portugal,110,1,POR,4,12.0,1,30.0,1,10,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,2.0,1.0,"[{'mdId': '2', 'tId': '110', 'tSCode': 'Portug...","[{'mdId': '1', 'tId': '110', 'tSCode': 'Portug...",10.0,0.8,10.0,1.0,0.0,17.0,1.5,3.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,250076574,K. Mbappé,Kylian Mbappé,Kylian Mbappe,France,43,1,FRA,4,12.0,1,35.0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '2', 'tId': '43', 'tSCode': 'France'...","[{'mdId': '1', 'tId': '43', 'tSCode': 'France'...",2.0,0.2,2.0,1.0,0.0,9.0,0.9,2.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,250016833,H. Kane,Harry Kane,Harry Kane,England,39,1,ENG,4,11.5,1,36.0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '2', 'tId': '39', 'tSCode': 'England...","[{'mdId': '1', 'tId': '39', 'tSCode': 'England...",2.0,0.2,2.0,1.0,0.0,11.0,1.4,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,250002096,R. Lewandowski,Robert Lewandowski,Robert Lewandowski,Poland,109,1,POL,4,11.5,1,15.0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '2', 'tId': '109', 'tSCode': 'Poland...","[{'mdId': '1', 'tId': '109', 'tSCode': 'Poland...",2.0,0.2,2.0,1.0,0.0,5.0,0.7,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,250010802,R. Lukaku,Romelu Lukaku,Romelu Lukaku,Belgium,13,1,BEL,4,11.0,1,49.0,1,10,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,2.0,1.0,"[{'mdId': '2', 'tId': '13', 'tSCode': 'Belgium...","[{'mdId': '1', 'tId': '13', 'tSCode': 'Belgium...",10.0,0.9,10.0,1.0,0.0,22.0,2.2,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
df.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1596897000,Aug 08 2020 - 2:30pm,complete,NaN,Club Brugge,Sporting Charleroi,Lawrence Visser,1.0,0.0,0.0,2.05,1.24,0,1,1,0,0,0,NaN,78,10,2,2,0,3,0,1,1,2,1,17,8,4,4,13,4,-1,-1,50,50,2.05,1.23,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.61,3.85,5.45,1.33,2.00,3.65,7.00,2.00,1.74,Jan Breydelstadion (Brugge)
1,1596906000,Aug 08 2020 - 5:00pm,complete,NaN,Royal Antwerp FC,Royal Excel Mouscron,Nathan Verboomen,1.0,0.0,0.0,1.75,0.82,1,1,2,0,0,0,49,78,7,3,2,0,1,0,0,2,0,1,14,5,11,3,3,2,9,3,64,36,2.27,0.86,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.44,4.20,6.00,1.22,1.70,2.62,5.00,1.80,1.95,Bosuilstadion (Deurne)
2,1596906000,Aug 08 2020 - 5:00pm,complete,NaN,Standard Liège,Cercle Brugge,Jonathan Lardot,1.0,0.0,0.0,1.55,1.00,1,0,1,0,0,0,55,NaN,5,4,1,0,1,0,0,1,0,1,18,8,10,3,8,5,-1,-1,50,50,2.40,1.24,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.44,4.40,6.75,1.22,1.71,2.80,5.15,1.83,1.87,Stade Maurice Dufrasne (Liège (Luik))
3,1596972600,Aug 09 2020 - 11:30am,complete,NaN,Sint-Truiden,KAA Gent,Jan Boterberg,1.0,0.0,0.0,1.24,1.40,2,1,3,2,1,1,"2,60",43,6,3,2,0,1,0,1,1,1,0,11,14,8,4,3,10,19,13,42,58,1.72,1.67,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,4.85,4.20,1.61,1.19,1.62,2.60,4.60,1.67,2.10,Stayen (Sint-Truiden (St.-Trond))
4,1596981600,Aug 09 2020 - 2:00pm,complete,NaN,Zulte-Waregem,KRC Genk,Erik Lambrechts,1.0,0.0,0.0,1.18,1.65,1,2,3,1,1,0,41,"73,79",9,8,3,0,1,0,0,3,0,1,10,15,7,9,3,6,-1,-1,50,50,1.51,1.93,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.70,3.85,1.87,1.15,1.50,2.25,3.90,1.48,2.50,Regenboogstadion (Waregem)


In [12]:
df_players.head()

,full_name,age,birthday,birthday_GMT,league,season,position,Current Club,minutes_played_overall,minutes_played_home,minutes_played_away,nationality,appearances_overall,appearances_home,appearances_away,goals_overall,goals_home,goals_away,assists_overall,assists_home,assists_away,penalty_goals,penalty_misses,clean_sheets_overall,clean_sheets_home,clean_sheets_away,conceded_overall,conceded_home,conceded_away,yellow_cards_overall,red_cards_overall,goals_involved_per_90_overall,assists_per_90_overall,goals_per_90_overall,goals_per_90_home,goals_per_90_away,min_per_goal_overall,conceded_per_90_overall,min_per_conceded_overall,min_per_match,min_per_card_overall,min_per_assist_overall,cards_per_90_overall,rank_in_league_top_attackers,rank_in_league_top_midfielders,rank_in_league_top_defenders,rank_in_club_top_scorer
0,Aaron Tshibola,26,789004800,1995/01/02,Pro League,2019/2020,Midfielder,Waasland-Beveren,692,258,434,Congo DR,8,3,5,0,0,0,0,0,0,0,0,0,0,0,16,6,10,2,0,0.00,0.00,0.00,0.00,0.00,0,2.08,43,87,346,0,0.26,236,261,-1,15
1,Abdoulaye Seck,28,707616000,1992/06/04,Pro League,2019/2020,Defender,Royal Antwerp FC,946,584,362,Senegal,13,8,5,0,0,0,0,0,0,0,0,3,2,1,9,4,5,2,0,0.00,0.00,0.00,0.00,0.00,0,0.86,105,73,473,0,0.19,329,276,20,21
2,Abdoulaye Sissako,22,896140800,1998/05/26,Pro League,2019/2020,Midfielder,Zulte-Waregem,1264,645,619,France,20,10,10,1,1,0,2,0,2,0,0,5,4,1,22,7,15,2,0,0.21,0.14,0.07,0.14,0.00,1264,1.57,57,63,632,632,0.14,164,80,-1,11
3,Abdul Ajagun,28,729302400,1993/02/10,Pro League,2019/2020,Midfielder,KV Kortrijk,844,420,424,Nigeria,18,10,8,1,0,1,2,1,1,0,0,3,3,0,17,10,7,1,0,0.32,0.21,0.11,0.00,0.21,844,1.81,50,47,844,422,0.11,137,52,-1,15
4,Abdul Manaf Nurudeen,22,918432000,1999/02/08,Pro League,2019/2020,Goalkeeper,AS Eupen,0,0,0,Ghana,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0.00,0,0,0,0,0.00,-1,-1,-1,-1


## Players matching with Euro data and Footystats data

In [13]:
df_players_euro['full_name'] = df_players_euro['pFName'].values

In [14]:
df_players_euro

,id,pDName,pFName,latinName,tName,tId,teamPlayed,cCode,skill,value,isActive,selPer,mdId,totPts,gS,assist,cS,gC,yC,rC,oG,pS,pC,pE,saves,pM,bR,gOB,mOM,mOMPts,pStatus,matchAtd,trained,isPlayed,selInPer,selOutPer,upcomingMatchesList,currentMatchesList,avgPlayerPts,avgPlayerValue,lastGdPoints,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10,category11,category12,category13,category14,category15,full_name
0,63706,C. Ronaldo,Cristiano Ronaldo,Cristiano Ronaldo,Portugal,110,1,POR,4,12.0,1,30.0,1,10,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,2.0,1.0,"[{'mdId': '2', 'tId': '110', 'tSCode': 'Portug...","[{'mdId': '1', 'tId': '110', 'tSCode': 'Portug...",10.0,0.8,10.0,1.0,0.0,17.0,1.5,3.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Cristiano Ronaldo
1,250076574,K. Mbappé,Kylian Mbappé,Kylian Mbappe,France,43,1,FRA,4,12.0,1,35.0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '2', 'tId': '43', 'tSCode': 'France'...","[{'mdId': '1', 'tId': '43', 'tSCode': 'France'...",2.0,0.2,2.0,1.0,0.0,9.0,0.9,2.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Kylian Mbappé
2,250016833,H. Kane,Harry Kane,Harry Kane,England,39,1,ENG,4,11.5,1,36.0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '2', 'tId': '39', 'tSCode': 'England...","[{'mdId': '1', 'tId': '39', 'tSCode': 'England...",2.0,0.2,2.0,1.0,0.0,11.0,1.4,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Harry Kane
3,250002096,R. Lewandowski,Robert Lewandowski,Robert Lewandowski,Poland,109,1,POL,4,11.5,1,15.0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '2', 'tId': '109', 'tSCode': 'Poland...","[{'mdId': '1', 'tId': '109', 'tSCode': 'Poland...",2.0,0.2,2.0,1.0,0.0,5.0,0.7,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Robert Lewandowski
4,250010802,R. Lukaku,Romelu Lukaku,Romelu Lukaku,Belgium,13,1,BEL,4,11.0,1,49.0,1,10,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,2.0,1.0,"[{'mdId': '2', 'tId': '13', 'tSCode': 'Belgium...","[{'mdId': '1', 'tId': '13', 'tSCode': 'Belgium...",10.0,0.9,10.0,1.0,0.0,22.0,2.2,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Romelu Lukaku
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,250117658,R. Colwill,Rubin Colwill,Rubin Colwill,Wales,144,1,WAL,3,4.0,1,10.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,0.0,"[{'mdId': '2', 'tId': '144', 'tSCode': 'Wales'...","[{'mdId': '1', 'tId': '144', 'tSCode': 'Wales'...",0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Rubin Colwill
616,72525,J. Toivio,Joona Toivio,Joona Toivio,Finland,42,1,FIN,2,4.0,1,1.0,1,6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,0.0,0.0,"[{'mdId': '2', 'tId': '42', 'tSCode': 'Finland...","[{'mdId': '1', 'tId': '42', 'tSCode': 'Finland...",6.0,1.5,6.0,0.0,0.0,0.0,0.1,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Joona Toivio
617,1908198,L. Négo,Loïc Négo,Loic Nego,Hungary,57,1,HUN,3,4.0,1,34.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,2.0,2.0,"[{'mdId': '2', 'tId': '57', 'tSCode': 'Hungary...","[{'mdId': '1', 'tId': '57', 'tSCode': 'Hungary...",0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Loïc Négo
618,250006714,M. Bizot,Marco Bizot,Marco Bizot,Netherlands,95,1,NED,1,4.0,1,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,0.0,0.0,"[{'mdId': '2', 'tId': '95', 'tSCode': 'Netherl...","[{'mdId': '1', 'tId': '95', 'tSCode': 'Netherl...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Marco Bizot


In [15]:
list_players_exist = []
list_players_euro_exist = []
for full_name in tqdm(df_players_euro['full_name'].values):
    base = r'^{}'
    expr = '(?=.*{})'
    words = full_name.split()  # example
    expr = base.format(''.join(expr.format(w) for w in words))
    df_name = df_players[df_players['full_name'].str.contains(expr)]
    if len(df_name) > 0:
        name = df_name['full_name'].values[0]
        list_players_euro_exist.append(full_name)
        list_players_exist.append(name)

100%|████████████████████████████████████████████████████████████████████████████████| 620/620 [00:43<00:00, 14.25it/s]


In [16]:
# players_exist = df_players_euro[df_players_euro['full_name'].isin(df_players['full_name'].unique())]['full_name'].values

In [17]:
df_players_exist = pd.DataFrame()
df_players_exist['full_name'] = list_players_euro_exist
df_players_exist['full_name1'] = list_players_exist

In [18]:
df_players = df_players[df_players['full_name'].isin(list_players_exist)]
df_players_euro = df_players_euro[df_players_euro['full_name'].isin(list_players_euro_exist)]

In [19]:
df_players_euro = pd.merge(df_players_euro[['full_name', 'tName', 'skill']], df_players_exist, on='full_name').rename(columns={'full_name': 'real_name', 'full_name1': 'full_name'})
df_players_euro

,real_name,tName,skill,full_name
0,Cristiano Ronaldo,Portugal,4,Cristiano Ronaldo
1,Kylian Mbappé,France,4,Kylian Mbappé
2,Harry Kane,England,4,Harry Kane
3,Robert Lewandowski,Poland,4,Robert Lewandowski
4,Romelu Lukaku,Belgium,4,Romelu Lukaku
...,...,...,...,...
563,Tomáš Koubek,Czech Republic,1,Tomáš Koubek
564,Rubin Colwill,Wales,3,Rubin Colwill
565,Joona Toivio,Finland,2,Joona Toivio
566,Marco Bizot,Netherlands,1,Marco Bizot


In [20]:
df_players = pd.merge(df_players, df_players_euro[['full_name', 'skill']], on='full_name')

In [21]:
df_players_euro['tName'].value_counts()

France             26
Germany            26
Croatia            26
Italy              26
Sweden             25
Slovakia           25
Belgium            25
Denmark            25
Netherlands        25
Austria            25
Switzerland        25
England            25
Portugal           24
Wales              24
Czech Republic     24
Scotland           24
Finland            23
Turkey             22
Poland             22
Hungary            22
Ukraine            22
Russia             20
North Macedonia    19
Spain              18
Name: tName, dtype: int64

In [22]:
df_players

,full_name,age,birthday,birthday_GMT,league,season,position,Current Club,minutes_played_overall,minutes_played_home,minutes_played_away,nationality,appearances_overall,appearances_home,appearances_away,goals_overall,goals_home,goals_away,assists_overall,assists_home,assists_away,penalty_goals,penalty_misses,clean_sheets_overall,clean_sheets_home,clean_sheets_away,conceded_overall,conceded_home,conceded_away,yellow_cards_overall,red_cards_overall,goals_involved_per_90_overall,assists_per_90_overall,goals_per_90_overall,goals_per_90_home,goals_per_90_away,min_per_goal_overall,conceded_per_90_overall,min_per_conceded_overall,min_per_match,min_per_card_overall,min_per_assist_overall,cards_per_90_overall,rank_in_league_top_attackers,rank_in_league_top_midfielders,rank_in_league_top_defenders,rank_in_club_top_scorer,skill
0,Eduard Sobol,26,798336000,1995/04/20,Pro League,2019/2020,Defender,Club Brugge,2119,1260,859,Ukraine,26,15,11,0,0,0,3,2,1,0,0,14,10,4,10,3,7,4,0,0.13,0.13,0.00,0.00,0.00,0,0.42,212,82,530,706,0.17,312,87,3,22,2
1,Eduard Sobol,26,798336000,1995/04/20,Pro League,2020/2021,Defender,Club Brugge,2165,1045,1120,Ukraine,30,14,16,2,1,1,4,1,3,0,0,9,4,5,22,10,12,6,0,0.25,0.17,0.08,0.09,0.08,1083,0.91,98,72,361,541,0.25,176,82,13,10,2
2,Eduard Sobol,26,798336000,1995/04/20,UEFA Champions League,2019/2020,Defender,Club Brugge,506,260,246,Ukraine,6,3,3,0,0,0,0,0,0,0,0,2,1,1,10,6,4,1,0,0.00,0.00,0.00,0.00,0.00,0,1.78,51,84,506,0,0.18,539,568,204,22,2
3,Eduard Sobol,26,798336000,1995/04/20,UEFA Champions League,2020/2021,Defender,Club Brugge,312,183,129,Ukraine,5,3,2,0,0,0,0,0,0,0,0,1,1,0,7,4,3,2,1,0.00,0.00,0.00,0.00,0.00,0,2.02,45,62,156,0,0.58,309,438,172,15,2
4,Eduard Sobol,25,798332400,1995/04/19,UEFA Europa League,2017/2018,Defender,Slavia Praha,540,270,270,Ukraine,6,3,3,0,0,0,2,1,1,0,0,2,1,1,6,3,3,2,0,0.33,0.33,0.00,0.00,0.00,0,1.00,90,90,270,270,0.33,530,48,250,16,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,Tomáš Suslov,18,1023408000,2002/06/07,Eredivisie,2019/2020,Forward,Groningen,3,3,0,Slovakia,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0.00,0,3,0,0,0.00,-1,-1,-1,15,3
6152,Tomáš Suslov,18,1023408000,2002/06/07,Eredivisie,2020/2021,Forward,Groningen,1492,653,839,Slovakia,29,11,18,2,1,1,5,3,2,0,0,8,5,3,21,8,13,1,0,0.42,0.30,0.12,0.14,0.11,746,1.27,71,51,1492,298,0.06,148,23,-1,9,3
6153,Vladimír Weiss,31,628387200,1989/11/30,International Friendlies,2021,Forward,Slovakia,84,57,27,Slovakia,2,1,1,0,0,0,1,1,0,0,0,1,0,1,1,1,0,0,0,1.07,1.07,0.00,0.00,0.00,0,1.07,84,42,0,84,0.00,-1,-1,-1,23,3
6154,Vladimír Weiss,31,628387200,1989/11/30,UEFA Euro Championship,2020,Forward,Slovakia,35,21,14,Slovakia,2,1,1,0,0,0,0,0,0,0,0,0,0,0,2,1,1,1,0,0.00,0.00,0.00,0.00,0.00,0,5.14,18,18,35,0,2.57,-1,-1,-1,6,3


In [23]:
df_players['league'].value_counts()

International Friendlies    1029
UEFA Champions League        848
UEFA Nations League          837
UEFA Europa League           745
UEFA Euro Championship       542
UEFA Euro Qualifiers         439
Premier League               372
Bundesliga                   335
Serie A                      268
FA Cup                       230
La Liga                      154
World Cup                    145
Ligue 1                      100
Eredivisie                    40
Copa del Rey                  32
Pro League                    22
Liga NOS                      18
Name: league, dtype: int64

## Putting some weights on attacking and defend point those who join top competition

In [24]:
list_top_league = ['UEFA Champions League', 'UEFA Europa League', 'UEFA Euro Championship']

In [25]:
list_top_league = ['UEFA Euro Championship']

In [26]:
df_players.loc[(df_players['season'].isin(['2020','2021','2020/2021'])) & (df_players['league'].isin(list_top_league)), 'bonus_attacking_point'] = 1.3
df_players.loc[(df_players['season'].isin(['2020','2021','2020/2021'])) & (df_players['league'].isin(list_top_league)), 'bonus_defending_point'] = 0.7
df_players.fillna(1, inplace=True)

In [27]:
df_players['goals_overall'] = df_players['goals_overall'] * df_players['bonus_attacking_point']
df_players['assists_overall'] = df_players['assists_overall'] * df_players['bonus_attacking_point']
df_players['goals_involved_per_90_overall'] = df_players['goals_involved_per_90_overall'] * df_players['bonus_attacking_point']
df_players['clean_sheets_overall'] = df_players['clean_sheets_overall'] * df_players['bonus_attacking_point']
df_players['conceded_overall'] = df_players['conceded_overall'] * df_players['bonus_defending_point']
df_players['conceded_per_90_overall'] = df_players['conceded_per_90_overall'] * df_players['bonus_defending_point']

In [28]:
def get_players(team):
    players = df_players[df_players['Current Club'] == team]['full_name'].unique()
    df_team_players = df_players[df_players['full_name'].isin(players)]
    return df_team_players

In [29]:
team = 'France'
df_team_country = get_players(team)

df_team_country_attacking = df_team_country[df_team_country['position'].isin(['Forward', 'Midfielder'])][['full_name', 'league', 'season',
       'Current Club', 'appearances_overall', 'goals_overall','assists_overall',
       'goals_involved_per_90_overall', 'skill']]

df_team_country_attacking = df_team_country_attacking.groupby(['full_name']).agg({'appearances_overall': 'sum',
                                                                                  'goals_overall': 'sum',
                                                                                 'assists_overall': 'sum',
                                                                                 'goals_involved_per_90_overall':'mean',
                                                                                 'skill': 'mean'}).sort_values('goals_involved_per_90_overall', ascending=False).reset_index()

df_team_country_attacking_score = pd.DataFrame(df_team_country_attacking.head().mean()).transpose()
df_team_country_attacking_score.columns = [i+'_attacking' for i in df_team_country_attacking_score.columns]
df_team_country_attacking_score
df_team_country_defending = df_team_country[df_team_country['position'].isin(['Defender', 'Midfielder', 'Goalkeeper'])][['full_name', 'league', 'season',
       'Current Club', 'appearances_overall','clean_sheets_overall', 'conceded_overall', 'conceded_per_90_overall', 'skill']]

df_team_country_defending = df_team_country_defending.groupby(['full_name']).agg({'appearances_overall': 'sum',
                                                                                  'clean_sheets_overall': 'sum',
                                                                                 'conceded_overall': 'sum',
                                                                                 'conceded_per_90_overall':'mean',
                                                                                 'skill': 'mean'}).sort_values('clean_sheets_overall', ascending=False).reset_index()

df_team_country_defending_score = pd.DataFrame(df_team_country_defending.head().mean()).transpose()
df_team_country_defending_score.columns = [i+'_defending' for i in df_team_country_defending_score.columns]

df_team_country_penalty = df_team_country[['full_name', 'league', 'season',
       'Current Club', 'penalty_goals']]

df_team_country_penalty = df_team_country_penalty.groupby(['full_name']).agg({'penalty_goals': 'mean'}).sort_values('penalty_goals', ascending=False).reset_index()

df_team_country_penalty_score = pd.DataFrame(df_team_country_penalty.head().mean()).transpose()
df_team_country_penalty_score.columns = [i+'_penalty' for i in df_team_country_penalty_score.columns]

df_team_score = df_team_country_attacking_score.join(df_team_country_defending_score).join(df_team_country_penalty_score)
df_team_score['country'] = team

In [30]:
df_team_country_defending = df_team_country[df_team_country['position'].isin(['Defender', 'Midfielder', 'Goalkeeper'])][['full_name', 'league', 'season',
       'Current Club', 'appearances_overall','clean_sheets_overall', 'conceded_overall', 'conceded_per_90_overall', 'skill']]
df_team_country_defending = df_team_country_defending.groupby(['full_name']).agg({'appearances_overall': 'sum',
                                                                                  'clean_sheets_overall': 'sum',
                                                                                 'conceded_overall': 'sum',
                                                                                 'conceded_per_90_overall':'mean',
                                                                                 'skill': 'mean'}).sort_values('clean_sheets_overall', ascending=False).reset_index()

df_team_country_defending

,full_name,appearances_overall,clean_sheets_overall,conceded_overall,conceded_per_90_overall,skill
0,N'Golo Kanté,182,77.3,149.1,0.714706,3
1,Raphaël Varane,186,70.3,156.1,0.821875,2
2,Clément Lenglet,175,67.0,174.0,0.886000,2
3,Thomas Lemar,146,66.0,76.0,0.615714,3
4,Jules Koundé,153,64.0,125.4,0.822000,2
5,Lucas Hernández Pi,132,63.3,72.7,0.609500,2
6,Hugo Lloris,182,62.3,186.1,1.171176,1
7,Presnel Kimpembe,141,60.3,119.1,0.693333,2
8,Benjamin Pavard,160,59.3,170.7,0.793571,2
9,Moussa Sissoko,164,59.0,122.0,0.897333,3


In [31]:
df_team_score

,appearances_overall_attacking,goals_overall_attacking,assists_overall_attacking,goals_involved_per_90_overall_attacking,skill_attacking,appearances_overall_defending,clean_sheets_overall_defending,conceded_overall_defending,conceded_per_90_overall_defending,skill_defending,penalty_goals_penalty,country
0,168.8,84.38,35.8,0.858167,4.0,168.4,68.92,136.12,0.772059,2.4,0.853781,France


In [32]:
def get_score_country(team):
    df_team_country = get_players(team)

    df_team_country_attacking = df_team_country[df_team_country['position'].isin(['Forward', 'Midfielder'])][['full_name', 'league', 'season',
           'Current Club', 'appearances_overall', 'goals_overall','assists_overall',
           'goals_involved_per_90_overall', 'skill']]

    df_team_country_attacking = df_team_country_attacking.groupby(['full_name']).agg({'appearances_overall': 'sum',
                                                                                      'goals_overall': 'sum',
                                                                                     'assists_overall': 'sum',
                                                                                     'goals_involved_per_90_overall':'mean',
                                                                                     'skill': 'mean'}).sort_values('goals_involved_per_90_overall', ascending=False).reset_index()

    df_team_country_attacking_score = pd.DataFrame(df_team_country_attacking.head().mean()).transpose()
    df_team_country_attacking_score.columns = [i+'_attacking' for i in df_team_country_attacking_score.columns]

    df_team_country_defending = df_team_country[df_team_country['position'].isin(['Defender', 'Midfielder', 'Goalkeeper'])][['full_name', 'league', 'season',
           'Current Club', 'appearances_overall','clean_sheets_overall', 'conceded_overall', 'conceded_per_90_overall', 'skill']]

    df_team_country_defending = df_team_country_defending.groupby(['full_name']).agg({'appearances_overall': 'sum',
                                                                                      'clean_sheets_overall': 'sum',
                                                                                     'conceded_overall': 'sum',
                                                                                     'conceded_per_90_overall':'mean',
                                                                                     'skill': 'mean'}).sort_values('clean_sheets_overall', ascending=False).reset_index()

    df_team_country_defending_score = pd.DataFrame(df_team_country_defending.head().mean()).transpose()
    df_team_country_defending_score.columns = [i+'_defending' for i in df_team_country_defending_score.columns]
    
    df_team_country_penalty = df_team_country[['full_name', 'league', 'season',
           'Current Club', 'penalty_goals']]

    df_team_country_penalty = df_team_country_penalty.groupby(['full_name']).agg({'penalty_goals': 'mean'}).sort_values('penalty_goals', ascending=False).reset_index()

    df_team_country_penalty_score = pd.DataFrame(df_team_country_penalty.head().mean()).transpose()
    df_team_country_penalty_score.columns = [i+'_penalty' for i in df_team_country_penalty_score.columns]

    df_team_score = df_team_country_attacking_score.join(df_team_country_defending_score).join(df_team_country_penalty_score)
    df_team_score['country'] = team
    return df_team_score

In [33]:
df_all_scores = pd.DataFrame()

for country in df_players_euro['tName'].unique():
    df_temp = get_score_country(country)
    if df_temp.isna().sum().sum() == 0:
        df_all_scores = df_all_scores.append(df_temp)

In [34]:
df_all_scores = df_all_scores[['country']+[i for i in df_all_scores.columns if 'country' not in i]].sort_values('conceded_per_90_overall_defending')

df_all_scores['probability_scoring'] = (df_all_scores['goals_overall_attacking'] + df_all_scores['goals_overall_attacking']) / df_all_scores['appearances_overall_attacking'] * df_all_scores['goals_involved_per_90_overall_attacking'] * (df_all_scores['skill_attacking'] / 4)
df_all_scores['probability_concede'] = ((df_all_scores['clean_sheets_overall_defending'] / df_all_scores['appearances_overall_defending']) - (df_all_scores['conceded_overall_defending'] / df_all_scores['appearances_overall_defending'])) * df_all_scores['conceded_per_90_overall_defending'] * (df_all_scores['skill_defending'] / 4)
df_all_scores['probability_concede'] = df_all_scores['probability_concede'].abs()

df_all_scores_final = df_all_scores[['country', 'penalty_goals_penalty', 'probability_scoring', 'probability_concede']].reset_index(drop=True)
df_all_scores_final['country'] = df_all_scores_final['country'].str.lower()
df_all_scores_final.sort_values('probability_scoring')

,country,penalty_goals_penalty,probability_scoring,probability_concede
20,slovakia,0.313376,0.084782,0.470504
17,finland,0.179365,0.124279,0.358190
15,russia,0.090909,0.124962,0.353367
16,czech republic,0.088571,0.135645,0.542686
12,hungary,0.096515,0.148401,0.441771
3,sweden,0.482500,0.179577,0.273250
6,wales,0.055385,0.182363,0.199922
18,scotland,0.082051,0.187146,0.487972
14,switzerland,0.100595,0.218571,0.421807
9,denmark,0.196233,0.260220,0.262466


## International matches

In [35]:
df_international_matches = pd.read_csv('data/results.csv')

df_international_matches

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
42100,2021-05-25,Indonesia,Afghanistan,2,3,Friendly,Dubai,United Arab Emirates,True
42101,2021-05-25,Oman,Thailand,1,0,Friendly,Dubai,United Arab Emirates,True
42102,2021-05-27,Turkey,Azerbaijan,2,1,Friendly,Alanya,Turkey,False
42103,2021-05-28,Bahrain,Malaysia,2,0,Friendly,Riffa,Bahrain,False


In [36]:
df_matches = df[(df['home_team_name'].isin(df_players_euro['tName'].unique())) | (df['away_team_name'].isin(df_players_euro['tName'].unique()))].reset_index(drop=True)

df_matches['datetime'] = pd.to_datetime(df_matches['timestamp'], unit='s')

df_matches = df_matches.sort_values('datetime')

df_info = df_matches[df_matches['timestamp'] != 1622314800][['datetime', 'home_team_name',
       'away_team_name', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count']]

df_info.rename(columns={'home_team_name': 'home_team',
       'away_team_name': 'away_team', 'home_team_goal_count': 'home_score',
       'away_team_goal_count' : 'away_score', 'total_goal_count': 'total_goals'}, inplace=True)

df_info = df_info[(df_info['datetime'] < '2021-06-25') & (df_info['datetime'] > '2021-05-29')]
df_info['date'] = df_info['datetime'].dt.date

In [37]:
df_info = df_international_matches[['date', 'home_team', 'away_team', 'home_score', 'away_score']].append(df_info[['date', 'home_team', 'away_team', 'home_score', 'away_score',]])
df_info['total_goals'] = df_info['home_score'] + df_info['away_score']
df_info

,date,home_team,away_team,home_score,away_score,total_goals
0,1872-11-30,Scotland,England,0,0,0
1,1873-03-08,England,Scotland,4,2,6
2,1874-03-07,Scotland,England,2,1,3
3,1875-03-06,England,Scotland,2,2,4
4,1876-03-04,Scotland,England,3,0,3
...,...,...,...,...,...,...
211,2021-06-23,Slovakia,Spain,0,5,5
177,2021-06-23,Portugal,France,0,0,0
214,2021-06-23,Germany,Hungary,2,2,4
213,2021-06-23,Portugal,France,2,2,4


In [38]:
df_info['date'] = df_info['date'].astype(str)

In [39]:
# removing duplicates in euro dataset
df_info = df_info.sort_values(['date', 'home_team', 'away_team', 'total_goals'], ascending=[True, True, True, False]).drop_duplicates(['date', 'home_team', 'away_team'])

In [40]:
df_info['home_team'] = df_info['home_team'].str.lower()
df_info['away_team'] = df_info['away_team'].str.lower()

In [41]:
# avg_home_goal_scored = data[(data['home_team'] == 'manchester city')].home_score.mean()

# avg_away_goal_conceded = data[(data['away_team'] == 'chelsea')].home_score.mean()

#  int(stats.mode(np.random.poisson(1/2*(avg_away_goal_conceded+avg_home_goal_scored),10000))[0])

# 1/2* (data[(data['home_team'] == 'manchester city')].home_score.mean() + data[(data['away_team'] == 'chelsea')].home_score.mean())

In [42]:
def get_probabilty_scoring(home_team, away_team):
    df_home = df_all_scores_final[df_all_scores_final['country'] == home_team]
    df_away = df_all_scores_final[df_all_scores_final['country'] == away_team]
    
    if len(df_home) > 0:
        if len(df_away) > 0:
            home_scoring = df_home['probability_scoring'].values[0] + df_away['probability_concede'].values[0]
            away_scoring = df_away['probability_scoring'].values[0] + df_home['probability_concede'].values[0]
        else:
            home_scoring = df_home['probability_scoring'].values[0] + 1
            away_scoring = 0 + df_home['probability_concede'].values[0]
    else:
        if len(df_away) > 0:
            home_scoring = 0 + df_away['probability_concede'].values[0]
            away_scoring = df_away['probability_scoring'].values[0] + 1
        else:
            home_scoring = 0
            away_scoring = 0
        
    
    return home_scoring, away_scoring

In [43]:
def get_probabilty_penalty(home_team, away_team):
    df_home = df_all_scores_final[df_all_scores_final['country'] == home_team]
    df_away = df_all_scores_final[df_all_scores_final['country'] == away_team]
    
    if len(df_home) > 0:
        if len(df_away) > 0:
            home_scoring = df_home['penalty_goals_penalty'].values[0]
            away_scoring = df_away['penalty_goals_penalty'].values[0]
        else:
            home_scoring = df_home['penalty_goals_penalty'].values[0]
            away_scoring = 0
    else:
        if len(df_away) > 0:
            home_scoring = 0
            away_scoring = df_away['penalty_goals_penalty'].values[0]
        else:
            home_scoring = 0
            away_scoring = 0
    
    return home_scoring, away_scoring

In [44]:
def round_school(x):
    i, f = divmod(x, 1)
    return int(i + ((f >= 0.5) if (x > 0) else (f > 0.5)))

In [53]:
def predict_score(home_team, away_team, extra_time_penalty):
    ht = (' '.join(home_team.split())).lower()
    at = (' '.join(away_team.split())).lower()
    
    if len(df_info[(df_info.home_team ==ht) & (df_info.away_team ==at)]) > 20:
        
        avg_home_score = df_info[(df_info.home_team ==ht) & (df_info.away_team ==at)].home_score.mean()
        avg_away_score = df_info[(df_info.home_team ==ht) & (df_info.away_team ==at)].away_score.mean()
        
#         home_goal = int(stats.mode(np.random.poisson(avg_home_score,10000))[0])                    
#         away_goal = int(stats.mode(np.random.poisson(avg_away_score,10000))[0])
        home_goal = np.mean(np.random.poisson(avg_home_score,10000))                
        away_goal = np.mean(np.random.poisson(avg_away_score,10000))
        
    else:
        avg_away_goal_conceded = df_info[(df_info.away_team ==at)].home_score.mean()
        avg_home_goal_scored   = df_info[(df_info.home_team ==ht)].home_score.mean()
#         home_goal = int(stats.mode(np.random.poisson(1/2*(avg_away_goal_conceded+avg_home_goal_scored),10000))[0])
        home_goal = np.mean(np.random.poisson(1/2*(avg_away_goal_conceded+avg_home_goal_scored),10000))
    
        avg_home_goal_conceded = df_info[(df_info.home_team ==ht)].away_score.mean()
        avg_away_goal_scored   = df_info[(df_info.away_team ==at)].away_score.mean()
#         away_goal = int(stats.mode(np.random.poisson(1/2*(avg_home_goal_conceded+avg_away_goal_scored),10000))[0])
        away_goal = np.mean(np.random.poisson(1/2*(avg_home_goal_conceded+avg_away_goal_scored),10000))
    
    prob_home_score, prob_away_score = get_probabilty_scoring(ht, at)
    prob_home_score_penalty, prob_away_score_penalty = get_probabilty_penalty(ht, at)
    
    home_goal = home_goal + prob_home_score + (prob_home_score_penalty * 0.1)
    away_goal = away_goal + prob_away_score + (prob_away_score_penalty * 0.1)
    
    home_goal = round_school(home_goal)
    away_goal = round_school(away_goal)
    
#     home_goal = int(home_goal + prob_home_score + (prob_home_score_penalty * 0.2))
#     away_goal = int(away_goal + prob_away_score + (prob_away_score_penalty * 0.2))
    
#     print(f'They have played {len(df_info[(df_info.home_team ==ht) & (df_info.away_team ==at)])} matches')
    
    if home_goal > away_goal:
        print(f'{home_team.capitalize()} win. The scoreline is {home_team.capitalize()} {home_goal}:{away_goal} {away_team.capitalize()}')
    elif home_goal < away_goal:
        print(f'{away_team.capitalize()} win. The scoreline is {home_team.capitalize()} {home_goal}:{away_goal} {away_team.capitalize()}')
    elif home_goal == away_goal:
        print(f'Draw. The scoreline is {home_team.capitalize()} {home_goal}:{away_goal} {away_team.capitalize()}')
        if extra_time_penalty:
            if prob_home_score_penalty > prob_away_score_penalty:
                print(f'Penalty won by {home_team.capitalize()}')
            elif prob_home_score_penalty < prob_away_score_penalty:
                print(f'Penalty won by {away_team.capitalize()}')
            else:
                if prob_home_score > prob_away_score:
                    print(f'Penalty won by {home_team.capitalize()}')
                else:
                    print(f'Penalty won by {away_team.capitalize()}')
        else:
            print(f'Draw. The scoreline is {home_team.capitalize()} {home_goal}:{away_goal} {away_team.capitalize()}')
    return home_goal, away_goal

In [47]:
def update_table(df_table, home_team, away_team, home_score, away_score):
    if home_team not in df_table['country'].values:
        df_temp = pd.DataFrame()
        df_temp['country'] = [home_team]
        if home_score > away_score:
            df_temp['win'] = [1]
            df_temp['draw'] = [0]
            df_temp['lose'] = [0]
            df_temp['gf'] = [home_score]
            df_temp['ga'] = [away_score]
            df_temp['pts'] = [3]
        elif home_score < away_score:
            df_temp['win'] = [0]
            df_temp['draw'] = [0]
            df_temp['lose'] = [1]
            df_temp['gf'] = [home_score]
            df_temp['ga'] = [away_score]
            df_temp['pts'] = [0]
        else:
            df_temp['win'] = [0]
            df_temp['draw'] = [1]
            df_temp['lose'] = [0]
            df_temp['gf'] = [home_score]
            df_temp['ga'] = [away_score]
            df_temp['pts'] = [1]
        df_table = df_table.append(df_temp)
    else:
        if home_score > away_score:
            df_temp = df_table[df_table['country'] == home_team]
            win = df_temp['win'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == home_team, 'win'] = win+1
            df_table.loc[df_table['country'] == home_team, 'gf'] = gf+home_score
            df_table.loc[df_table['country'] == home_team, 'ga'] = ga+away_score
            df_table.loc[df_table['country'] == home_team, 'pts'] = pts+3
        elif home_score < away_score:
            df_temp = df_table[df_table['country'] == home_team]
            lose = df_temp['lose'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == home_team, 'lose'] = lose+1
            df_table.loc[df_table['country'] == home_team, 'gf'] = gf+home_score
            df_table.loc[df_table['country'] == home_team, 'ga'] = ga+away_score
            df_table.loc[df_table['country'] == home_team, 'pts'] = pts+0
        else:
            df_temp = df_table[df_table['country'] == home_team]
            draw = df_temp['draw'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == home_team, 'draw'] = draw+1
            df_table.loc[df_table['country'] == home_team, 'gf'] = gf+home_score
            df_table.loc[df_table['country'] == home_team, 'ga'] = ga+away_score
            df_table.loc[df_table['country'] == home_team, 'pts'] = pts+1
            
    if away_team not in df_table['country'].values:
        df_temp = pd.DataFrame()
        df_temp['country'] = [away_team]
        if away_score > home_score:
            df_temp['win'] = [1]
            df_temp['draw'] = [0]
            df_temp['lose'] = [0]
            df_temp['gf'] = [away_score]
            df_temp['ga'] = [home_score]
            df_temp['pts'] = [3]
        elif away_score < home_score:
            df_temp['win'] = [0]
            df_temp['draw'] = [0]
            df_temp['lose'] = [1]
            df_temp['gf'] = [away_score]
            df_temp['ga'] = [home_score]
            df_temp['pts'] = [0]
        else:
            df_temp['win'] = [0]
            df_temp['draw'] = [1]
            df_temp['lose'] = [0]
            df_temp['gf'] = [away_score]
            df_temp['ga'] = [home_score]
            df_temp['pts'] = [1]
        df_table = df_table.append(df_temp)
    else:
        if home_score < away_score:
            df_temp = df_table[df_table['country'] == away_team]
            win = df_temp['win'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == away_team, 'win'] = win+1
            df_table.loc[df_table['country'] == away_team, 'gf'] = gf+away_score
            df_table.loc[df_table['country'] == away_team, 'ga'] = ga+home_score
            df_table.loc[df_table['country'] == away_team, 'pts'] = pts+3
        elif home_score > away_score:
            df_temp = df_table[df_table['country'] == away_team]
            lose = df_temp['lose'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == away_team, 'lose'] = lose+1
            df_table.loc[df_table['country'] == away_team, 'gf'] = gf+away_score
            df_table.loc[df_table['country'] == away_team, 'ga'] = ga+home_score
            df_table.loc[df_table['country'] == away_team, 'pts'] = pts+0
        else:
            df_temp = df_table[df_table['country'] == away_team]
            draw = df_temp['draw'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == away_team, 'draw'] = draw+1
            df_table.loc[df_table['country'] == away_team, 'gf'] = gf+home_score
            df_table.loc[df_table['country'] == away_team, 'ga'] = ga+away_score
            df_table.loc[df_table['country'] == away_team, 'pts'] = pts+1
    return df_table

In [48]:
df_matches_euro = df_matches_euro[['htName', 'atName', 'mdName', 'groupName']]
df_matches_euro.rename(columns={'htName': 'home_team', 'atName': 'away_team'}, inplace=True)
df_matches_euro

,home_team,away_team,mdName,groupName
0,Turkey,Italy,Match 1,Group A
1,Wales,Switzerland,Match 2,Group A
2,Denmark,Finland,Match 3,Group B
3,Belgium,Russia,Match 4,Group B
4,England,Croatia,Match 6,Group D
5,Austria,North Macedonia,Match 5,Group C
6,Netherlands,Ukraine,Match 7,Group C
7,Scotland,Czech Republic,Match 9,Group D
8,Poland,Slovakia,Match 8,Group E
9,Spain,Sweden,Match 10,Group E


In [49]:
df_matches_euro['home_team'] = df_matches_euro['home_team'].str.lower()
df_matches_euro['away_team'] = df_matches_euro['away_team'].str.lower()

In [50]:
dict_tables = {}

In [55]:
# this code is for group stage

# for index, data in df_matches_euro[:36].iterrows():
#     if data['groupName'] in dict_tables:
#         df_table = dict_tables[data['groupName']]
#         home_score, away_score = predict_score(data['home_team'], data['away_team'], False)
#         df_table = update_table(df_table, data['home_team'], data['away_team'], home_score, away_score)
#         dict_tables[data['groupName']] = df_table.sort_values('pts', ascending=False).reset_index(drop=True)
#     else:
#         df_table = pd.DataFrame(columns=['country', 'win', 'draw', 'lose', 'gf', 'ga', 'pts'])
#         home_score, away_score = predict_score(data['home_team'], data['away_team'], False)
#         df_table = update_table(df_table, data['home_team'], data['away_team'], home_score, away_score)
#         dict_tables[data['groupName']] = df_table.sort_values('pts', ascending=False).reset_index(drop=True)

Draw. The scoreline is Wales 2:2 Denmark
Penalty won by Denmark
Italy win. The scoreline is Italy 3:2 Austria
Netherlands win. The scoreline is Netherlands 3:2 Czech republic
Belgium win. The scoreline is Belgium 3:2 Portugal
Draw. The scoreline is Croatia 2:2 Spain
Penalty won by Spain
France win. The scoreline is France 3:2 Switzerland
England win. The scoreline is England 3:2 Germany
Draw. The scoreline is Sweden 2:2 Ukraine
Penalty won by Sweden


In [47]:
from IPython.core import display as ICD

In [48]:
for group in sorted(list(dict_tables.keys())):
    print(group)
    ICD.display(dict_tables[group])

Group A


,country,win,draw,lose,gf,ga,pts
0,italy,2,1,0,9,4,7
1,turkey,0,3,0,6,6,3
2,wales,0,2,1,5,7,2
3,switzerland,0,2,1,5,8,2


Group B


,country,win,draw,lose,gf,ga,pts
0,belgium,2,1,0,8,6,7
1,denmark,1,2,0,8,5,5
2,russia,0,2,1,6,7,2
3,finland,0,1,2,5,9,1


Group C


...

,country,win,draw,lose,gf,ga,pts
0,england,3,0,0,9,6,9
1,croatia,1,1,1,7,7,4
2,czech republic,0,2,1,6,7,2
3,scotland,0,1,2,6,8,1


Group E


,country,win,draw,lose,gf,ga,pts
0,spain,2,1,0,8,6,7
1,poland,1,1,1,7,7,4
2,sweden,1,1,1,7,7,4
3,slovakia,0,1,2,6,8,1


Group F


,country,win,draw,lose,gf,ga,pts
0,portugal,2,1,0,8,6,7
1,france,2,0,1,8,7,6
2,germany,1,0,2,7,8,3
3,hungary,0,1,2,6,8,1


## Round of 16

In [57]:
for index, data in df_matches_euro[36:44].iterrows():
    if data['groupName'] in dict_tables:
        df_table = dict_tables[data['groupName']]
        home_score, away_score = predict_score(data['home_team'], data['away_team'], True)
        df_table = update_table(df_table, data['home_team'], data['away_team'], home_score, away_score)
        dict_tables[data['groupName']] = df_table.sort_values('pts', ascending=False).reset_index(drop=True)
    else:
        df_table = pd.DataFrame(columns=['country', 'win', 'draw', 'lose', 'gf', 'ga', 'pts'])
        home_score, away_score = predict_score(data['home_team'], data['away_team'], True)
        df_table = update_table(df_table, data['home_team'], data['away_team'], home_score, away_score)
        dict_tables[data['groupName']] = df_table.sort_values('pts', ascending=False).reset_index(drop=True)

Draw. The scoreline is Wales 2:2 Denmark
Penalty won by Denmark
Italy win. The scoreline is Italy 3:2 Austria
Netherlands win. The scoreline is Netherlands 3:2 Czech republic
Belgium win. The scoreline is Belgium 3:2 Portugal
Draw. The scoreline is Croatia 2:2 Spain
Penalty won by Spain
France win. The scoreline is France 3:2 Switzerland
England win. The scoreline is England 3:2 Germany
Draw. The scoreline is Sweden 2:2 Ukraine
Penalty won by Sweden


In [58]:
predict_score('wales', 'denmark', True)

Draw. The scoreline is Wales 2:2 Denmark
Penalty won by Denmark


(2, 2)

In [59]:
predict_score('italy', 'austria', True)

Italy win. The scoreline is Italy 3:2 Austria


(3, 2)

In [60]:
predict_score('netherlands', 'czech republic', True)

Netherlands win. The scoreline is Netherlands 3:2 Czech republic


(3, 2)

In [61]:
predict_score('belgium', 'portugal', True)

Belgium win. The scoreline is Belgium 3:2 Portugal


(3, 2)

In [62]:
predict_score('croatia', 'spain', True)

Draw. The scoreline is Croatia 2:2 Spain
Penalty won by Spain


(2, 2)

In [63]:
predict_score('france', 'switzerland', True)

France win. The scoreline is France 3:2 Switzerland


(3, 2)

In [64]:
predict_score('england', 'germany', True)

England win. The scoreline is England 3:2 Germany


(3, 2)

In [65]:
predict_score('sweden', 'ukraine', True)

Draw. The scoreline is Sweden 2:2 Ukraine
Penalty won by Sweden


(2, 2)

## Quarter Finals

In [66]:
predict_score('belgium', 'italy', True)

Draw. The scoreline is Belgium 2:2 Italy
Penalty won by Italy


(2, 2)

In [67]:
predict_score('france', 'spain', True)

France win. The scoreline is France 3:2 Spain


(3, 2)

In [68]:
predict_score('sweden', 'england', True)

Draw. The scoreline is Sweden 2:2 England
Penalty won by England


(2, 2)

In [69]:
predict_score('netherlands', 'denmark', True)

Netherlands win. The scoreline is Netherlands 3:2 Denmark


(3, 2)

## Semi Finals

In [70]:
predict_score('italy', 'france', True)

Italy win. The scoreline is Italy 3:2 France


(3, 2)

In [72]:
predict_score('england', 'netherlands', True)

England win. The scoreline is England 3:2 Netherlands


(3, 2)

## Finals

In [73]:
predict_score('italy', 'england', True)

Draw. The scoreline is Italy 2:2 England
Penalty won by Italy


(2, 2)

<img src="bracket_landscape.png">